In [1]:
import pandas as pd

# 파일 경로
file1 = '/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_file.csv'
file2 = '/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_auto_tune.csv'
output_path = '/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_all.csv'

# CSV 파일 읽기
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)

# 데이터 합치기
df_merged = pd.concat([df1, df2], ignore_index=True)

# 저장
df_merged.to_csv(output_path, index=False)
print(f"✅ 파일이 성공적으로 합쳐졌습니다: {output_path}")


✅ 파일이 성공적으로 합쳐졌습니다: /home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_all.csv


### subset 나누기

In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split

# 🔍 경로 및 데이터 불러오기
meta_path = "/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta.csv"
df = pd.read_csv(meta_path)

# 📌 분할 컬럼 추가
df['Split'] = None

# ✅ 1. Bonafide 분할
# 1-1. CommonVoice는 eval
df.loc[(df['label2'] == 'bonafide') & (df['group'] == 'TIMIT'), 'Split'] = 'eval'

# 1-2. VCTK & LibriSpeech → train/dev 분할
real_train_dev = df[(df['label2'] == 'bonafide') & (df['group'].isin(['VCTK', 'LibriSpeech']))]
train_idx, dev_idx = train_test_split(real_train_dev.index, test_size=0.3, random_state=42, shuffle=True)
df.loc[train_idx, 'Split'] = 'train'
df.loc[dev_idx, 'Split'] = 'dev'

# ✅ 2. Spoof 분할
spoof_df = df[df['label2'] == 'spoof']
for group_name in spoof_df['group'].unique():
    group_data = spoof_df[spoof_df['group'] == group_name]
    
    # 50% → eval
    group_eval_idx = group_data.sample(frac=0.5, random_state=42).index
    df.loc[group_eval_idx, 'Split'] = 'eval'
    
    # 나머지 → train/dev
    remaining = group_data.drop(index=group_eval_idx)
    group_train_idx, group_dev_idx = train_test_split(remaining.index, test_size=0.3, random_state=42)
    df.loc[group_train_idx, 'Split'] = 'train'
    df.loc[group_dev_idx, 'Split'] = 'dev'

# ✅ 검증
print(df['Split'].value_counts())

# 💾 저장
output_path = "/home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_subset.csv"
df.to_csv(output_path, index=False)
print(f"\n✅ 데이터셋 분할 완료: {output_path}")


train    270269
eval     152900
dev      115831
Name: Split, dtype: int64

✅ 데이터셋 분할 완료: /home/woongjae/noise-tracing/muti-feature_fusion/Datasets/meta_subset.csv
